In [1]:
import pandas as pd
import numpy as np
# https://www.kaggle.com/code/carlmcbrideellis/feature-selection-using-the-boruta-shap-package
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import random
import pickle

In [2]:
data = pd.read_csv("data_with_new_features.csv")
features = pd.read_csv('feature_importances_xgb.csv', nrows = 20)
features.head()
selected_features = list[features['Unnamed: 0'].values]
selected_features

list[array(['months_employed', 'credit_score', 'age', 'interest_rate',
       'credit_age_factor', 'dtiratio', 'interest_payment_burden',
       'income_to_interest_ratio', 'credit_income_inter',
       'income_to_loan_ratio', 'loan_pay_to_inc_ratio', 'loan_amount',
       'empl_sta', 'credit_util_ratio', 'income', 'has_co_signer=No',
       'employment_type=Full-time', 'loan_term',
       'employment_type=Unemployed', 'has_dependents=No'], dtype=object)]

In [3]:
data_new = data.copy()
y = data_new["default"]
del data_new["default"]

data_new = data_new[selected_features]

In [4]:
df_full_train, df_test, y_full_train, y_test = train_test_split(data_new, y, stratify = y, test_size = 0.2, random_state = 1)

df_train, df_val, y_train, y_val = train_test_split(df_full_train, y_full_train, stratify =y_full_train,  test_size = 0.25, random_state = 1)

In [5]:
df_full_train = df_full_train.reset_index(drop = True)
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [6]:
y_full_train = y_full_train.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [7]:
full_train_dicts = df_full_train.to_dict(orient = 'records')
train_dicts = df_train.to_dict(orient = 'records')
val_dicts = df_val.to_dict(orient = 'records')
test_dicts = df_test.to_dict(orient = 'records')

In [8]:
# I was getting a memory error, so I set sparse=True for the DictVectorizer.
dv = DictVectorizer(sparse = True)
dv.fit(train_dicts)
X_full_train_new = dv.transform(full_train_dicts)
X_train_new = dv.transform(train_dicts)
X_val_new = dv.transform(val_dicts)
X_test_new = dv.transform(test_dicts)

In [9]:



from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(class_weight='balanced', y=y_full_train)

In [10]:
# fit model no training data
model = XGBClassifier(params = {'subsample': 0.9, 'scale_pos_weight': 1, 'reg_lambda': 1,'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 5,
'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.9}, objective="binary:logistic", feature_name=dv.get_feature_names_out(), random_state = 1)
model.fit(X_full_train_new, y_full_train,sample_weight=sample_weights)

C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:43:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "feature_name", "params" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_name=array(['age', 'credit_age_factor', 'credit_income_inter', ...,
       'marital_status_dependents=Single_Yes', 'months_employed',
       'num_credit_lin...
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [11]:
y_pred = model.predict_proba(X_test_new)[:,1]

In [12]:
print(y_pred)

[0.75899124 0.67681766 0.54432464 ... 0.37157747 0.14638917 0.1884372 ]


In [13]:
y_pred > 0.5
y_decision = (y_pred >= 0.5).astype(int)
acc = (y_test == y_decision).mean().round(2)
print(f"Accuracy is calculated as \033[1m{acc}\033[0m.")
f_macro = f1_score(y_test, y_decision, average='macro').round(2)
print(f"Macro F1 score is calculated as \033[1m{f_macro}\033[0m.")
f_weighted = f1_score(y_test, y_decision, average='weighted').round(2)
print(f"Weighted F1 score is calculated as \033[1m{f_weighted}\033[0m.")

Accuracy is calculated as 0.71.
Macro F1 score is calculated as 0.58.
Weighted F1 score is calculated as 0.76.


In [14]:
seeds = [5, 13, 29, 43, 57, 87, 29, 42,1, 93]

In [31]:
#I initiannly get same results for each run. I tried the change subsample, colsample_bytree parameters but it did not help.
#To devaluate non deterministic results I st booster to gblinear readinf the xgboost documentation 
#https://readthedocs.org/projects/xgboost/downloads/pdf/latest/

results = []
for i in [5, 13, 29, 43, 57, 87, 29, 42,1, 93]:
    print(i)
    # I have deleted subsample and colsample_bytree parameters since I have evaluated same result at each run
    model = XGBClassifier(params = {'subsample': 0.9, 'scale_pos_weight': 1, 'reg_lambda': 1,'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 5,
'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.9}, objective="binary:logitraw", feature_selector= "shuffle", booster = 'gblinear',
                          feature_name=dv.get_feature_names_out(), random_state = i)
    model.fit(X_full_train_new, y_full_train,sample_weight=sample_weights)
    y_pred = model.predict_proba(X_test_new)[:,1]
    y_pred > 0.5
    y_decision = (y_pred >= 0.5).astype(int)
    acc = (y_test == y_decision).mean().round(4)
    print(f"Accuracy is calculated as \033[1m{acc}\033[0m.")
    f_macro = f1_score(y_test, y_decision, average='macro').round(4)
    print(f"Macro F1 score is calculated as \033[1m{f_macro}\033[0m.")
    f_weighted = f1_score(y_test, y_decision, average='weighted').round(4)
    print(f"Weighted F1 score is calculated as \033[1m{f_weighted}\033[0m.")
    results.append((acc,f_macro,f_weighted))
    output_file = f'modelxgb_i={i}.bin'
    with open(output_file, 'wb') as f_out:
        pickle.dump((dv, model), f_out)
    

5


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:37:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "feature_name", "params" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy is calculated as 0.7999.
Macro F1 score is calculated as 0.6245.
Weighted F1 score is calculated as 0.8215.
13


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:37:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "feature_name", "params" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy is calculated as 0.8004.
Macro F1 score is calculated as 0.6244.
Weighted F1 score is calculated as 0.8218.
29


...

Accuracy is calculated as 0.8005.
Macro F1 score is calculated as 0.6251.
Weighted F1 score is calculated as 0.822.
1


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:37:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "feature_name", "params" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy is calculated as 0.8.
Macro F1 score is calculated as 0.6247.
Weighted F1 score is calculated as 0.8216.
93


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:37:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "feature_name", "params" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy is calculated as 0.7999.
Macro F1 score is calculated as 0.6241.
Weighted F1 score is calculated as 0.8215.
